# Predicting Next Fixture Points Based on Past Performance

In [43]:
import pandas as pd
import re
import random
import requests
import pickle
import sklearn


## Get player data and gameweek data

Import packages

Get data from the FPL API

Code source: https://medium.com/analytics-vidhya/getting-started-with-fantasy-premier-league-data-56d3b9be8c32

In [13]:
# set url for fantasy PL API
api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# download the webpage
data = requests.get(api_url)

json = data.json()

json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [14]:
# build a dataframe
players = pd.DataFrame(json['elements'])

#players.columns

In [15]:
#select only relevant columns from elements_df, not all needed at this point but maybe useful in the future
#players_df_select = elements_df[['first_name','second_name','team','element_type','selected_by_percent',
#                                'now_cost','minutes','transfers_in','value_season','total_points']]

# use all columns 
players_df_select = players

# combine first and last names to get player full names
players_df_select['full_name'] = players_df_select[['first_name', 'second_name']].agg(' '.join, axis=1)

# drop first and last name columns
players_df_select = players_df_select.drop(['first_name', 'second_name'], axis = 1)

# player prices are 10x the true value. Divide the prices by 10 to get the true values
players_df_select['now_cost'] = players_df_select['now_cost']/10

players_df_select.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,full_name
0,0.0,0.0,232223,0,0,-1,1,0,4,0.0,...,90,524,55,576,60,293,48,0.00,0.00,Folarin Balogun
1,NaN,NaN,58822,0,0,-1,1,0,2,0.5,...,219,364,119,439,154,243,95,0.00,0.00,Cédric Alves Soares
2,100.0,100.0,153256,0,0,-1,1,0,3,0.5,...,275,500,182,558,211,430,139,0.00,0.00,Mohamed Elneny
3,100.0,100.0,438098,0,0,-1,1,0,3,1.2,...,87,291,137,118,51,376,119,0.97,0.49,Fábio Ferreira Vieira
4,100.0,100.0,226597,0,0,-3,3,0,2,5.5,...,41,34,12,104,36,33,11,0.95,0.57,Gabriel dos Santos Magalhães


In [16]:
# get team info
teams = pd.DataFrame(json['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1230,1285,1250,1250,1210,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1115,1175,1130,1190,1100,1160,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1060,1095,1050,1100,1060,1090,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1125,1205,1120,1220,1130,1190,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1165,1210,1120,1200,1210,1240,131


Get team defensive strength for each team

In [17]:
team_strength_def = teams[['id', 'name', 'strength_defence_away', 'strength_defence_home']]

team_strength_def

,id,name,strength_defence_away,strength_defence_home
0,1,Arsenal,1320,1210
1,2,Aston Villa,1160,1100
2,3,Bournemouth,1090,1060
3,4,Brentford,1190,1130
4,5,Brighton,1240,1210
5,6,Burnley,1080,1060
6,7,Chelsea,1110,1100
7,8,Crystal Palace,1085,1080
8,9,Everton,1080,1080
9,10,Fulham,1140,1100


Get team attack strength for each team

In [18]:
team_strength_att = teams[['id', 'name', 'strength_attack_away', 'strength_attack_home']]

team_strength_att

,id,name,strength_attack_away,strength_attack_home
0,1,Arsenal,1250,1250
1,2,Aston Villa,1190,1130
2,3,Bournemouth,1100,1050
3,4,Brentford,1220,1120
4,5,Brighton,1200,1120
5,6,Burnley,1080,1060
6,7,Chelsea,1210,1130
7,8,Crystal Palace,1170,1140
8,9,Everton,1120,1070
9,10,Fulham,1090,1090


Get position info

In [19]:
# get position information from 'element_types'
positions = pd.DataFrame(json['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],85
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],235
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],312
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],94


Merge the players with their teams

In [20]:
# merge player data with teams and positions
player_team_merge = pd.merge(
    left = players_df_select,
    right = teams,
    left_on = 'team',
    right_on = 'id'
)

player_team_merge[['full_name', 'name']].head()

,full_name,name
0,Folarin Balogun,Arsenal
1,Cédric Alves Soares,Arsenal
2,Mohamed Elneny,Arsenal
3,Fábio Ferreira Vieira,Arsenal
4,Gabriel dos Santos Magalhães,Arsenal


Merge the players with their positions

In [21]:
player_team_pos_merge = pd.merge(
    left = player_team_merge,
    right = positions,
    left_on = 'element_type',
    right_on = 'id'
)

player_team_pos_merge[['full_name', 'name', 'singular_name_short']].head()

,full_name,name,singular_name_short
0,Folarin Balogun,Arsenal,FWD
1,Gabriel Fernando de Jesus,Arsenal,FWD
2,Eddie Nketiah,Arsenal,FWD
3,Keinan Davis,Aston Villa,FWD
4,Jhon Durán,Aston Villa,FWD


In [22]:
# rename columns
player_team_pos_merge = player_team_pos_merge.rename(
    columns={'name':'team_name', 'singular_name_short':'position_name'}
)

player_team_pos_merge[['full_name', 'team_name', 'position_name']].head()

,full_name,team_name,position_name
0,Folarin Balogun,Arsenal,FWD
1,Gabriel Fernando de Jesus,Arsenal,FWD
2,Eddie Nketiah,Arsenal,FWD
3,Keinan Davis,Aston Villa,FWD
4,Jhon Durán,Aston Villa,FWD


Get player gameweek data from https://fantasy.premierleague.com/api/element-summary/

In [23]:
# function for getting specific player gameweek history
def get_history(player_id):
    ''' get all gameweek history for a given player'''
    
    # request data from API 
    data = requests.get("https://fantasy.premierleague.com/api/element-summary/" + str(player_id) + "/")
    json = data.json()
    
    # turn data into Pandas dataframe
    df = pd.DataFrame(json['history'])
    
    return df

get_history(1)  

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,1,2,16,0,True,2023-08-12T12:00:00Z,2,1,1,0,...,0,0.00,0.00,0.00,0.00,45,0,59090,0,0
1,1,12,8,0,False,2023-08-21T19:00:00Z,0,1,2,0,...,0,0.00,0.00,0.00,0.00,45,-4744,63768,4959,9703
2,1,21,10,0,True,2023-08-26T14:00:00Z,2,2,3,0,...,0,0.00,0.00,0.00,0.00,45,-7682,58109,2507,10189
3,1,31,14,0,True,2023-09-03T15:30:00Z,3,1,4,0,...,0,0.00,0.00,0.00,0.00,44,-9696,49814,2558,12254
4,1,43,9,0,False,2023-09-17T15:30:00Z,0,1,5,0,...,0,0.00,0.00,0.00,0.00,44,-12786,37048,0,12786
5,1,51,18,0,True,2023-09-24T13:00:00Z,2,2,6,0,...,0,0.00,0.00,0.00,0.00,44,-4527,32878,0,4527
6,1,62,3,0,False,2023-09-30T14:00:00Z,0,4,7,0,...,0,0.00,0.00,0.00,0.00,44,-3132,29679,0,3132
7,1,71,13,0,True,2023-10-08T15:30:00Z,1,0,8,0,...,0,0.00,0.00,0.00,0.00,44,-2349,27390,0,2349


In [24]:
from tqdm.auto import tqdm
tqdm.pandas()

/Users/elmeri/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

# rename columns
players = players.rename(
    columns={'name':'team', 'singular_name_short':'position'}
)

players.head()


,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,...,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,full_name,team,position
0,0.0,0.0,232223,0,0,-1,1,0,0.0,0.0,...,55,576,60,293,48,0.00,0.00,Folarin Balogun,Arsenal,FWD
1,100.0,100.0,205651,-1,1,-1,1,0,3.2,2.2,...,17,161,24,92,19,0.92,0.61,Gabriel Fernando de Jesus,Arsenal,FWD
2,NaN,NaN,205533,0,0,0,0,0,3.2,2.2,...,14,76,12,75,13,1.11,0.79,Eddie Nketiah,Arsenal,FWD
3,0.0,0.0,221239,0,0,-1,1,0,0.0,0.0,...,82,680,83,366,59,0.00,0.00,Keinan Davis,Aston Villa,FWD
4,100.0,100.0,476344,0,0,0,0,0,0.3,0.3,...,47,175,27,286,46,0.00,0.00,Jhon Durán,Aston Villa,FWD


In [26]:
# get gameweek history for all players
points = players['id_player'].progress_apply(get_history)

# combine results into one dataframe
points = pd.concat(df for df in points)

# join full_name
points = players[['id_player', 'full_name', 'team', 'position']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

  0%|          | 0/726 [00:00<?, ?it/s]

100%|██████████| 726/726 [04:15<00:00,  2.84it/s]


In [27]:
# merge opponent defensive strength
points = pd.merge(left = points,
                  right = team_strength_def[['id', 'strength_defence_away', 
                                             'strength_defence_home']],
                  how = 'left',
                  left_on = 'opponent_team',
                  right_on = 'id'
).drop(
    'id', axis = 1
).rename(
    columns={'strength_defence_away':'opp_def_strength_away', 'strength_defence_home':'opp_def_strength_home'}
)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength_away,opp_def_strength_home
0,1,Folarin Balogun,Arsenal,FWD,1,2,16,0,True,2023-08-12T12:00:00Z,...,0.00,0.00,0.00,45,0,59090,0,0,1120,1050
1,1,Folarin Balogun,Arsenal,FWD,1,12,8,0,False,2023-08-21T19:00:00Z,...,0.00,0.00,0.00,45,-4744,63768,4959,9703,1085,1080
2,1,Folarin Balogun,Arsenal,FWD,1,21,10,0,True,2023-08-26T14:00:00Z,...,0.00,0.00,0.00,45,-7682,58109,2507,10189,1140,1100
3,1,Folarin Balogun,Arsenal,FWD,1,31,14,0,True,2023-09-03T15:30:00Z,...,0.00,0.00,0.00,44,-9696,49814,2558,12254,1240,1140
4,1,Folarin Balogun,Arsenal,FWD,1,43,9,0,False,2023-09-17T15:30:00Z,...,0.00,0.00,0.00,44,-12786,37048,0,12786,1080,1080


In [28]:
# assign correct home/away opponent defensive strength for each fixture
def opp_def_strength(row):
    if row['was_home'] == False:
        return row['opp_def_strength_home']
    elif row['was_home'] == True:
        return row['opp_def_strength_away']
    else:
        return "Unknown"

points['opp_def_strength'] = points.apply(lambda row: opp_def_strength(row), axis = 1)

points = points.drop(['opp_def_strength_home','opp_def_strength_away'], axis = 1)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength
0,1,Folarin Balogun,Arsenal,FWD,1,2,16,0,True,2023-08-12T12:00:00Z,...,0.00,0.00,0.00,0.00,45,0,59090,0,0,1120
1,1,Folarin Balogun,Arsenal,FWD,1,12,8,0,False,2023-08-21T19:00:00Z,...,0.00,0.00,0.00,0.00,45,-4744,63768,4959,9703,1080
2,1,Folarin Balogun,Arsenal,FWD,1,21,10,0,True,2023-08-26T14:00:00Z,...,0.00,0.00,0.00,0.00,45,-7682,58109,2507,10189,1140
3,1,Folarin Balogun,Arsenal,FWD,1,31,14,0,True,2023-09-03T15:30:00Z,...,0.00,0.00,0.00,0.00,44,-9696,49814,2558,12254,1240
4,1,Folarin Balogun,Arsenal,FWD,1,43,9,0,False,2023-09-17T15:30:00Z,...,0.00,0.00,0.00,0.00,44,-12786,37048,0,12786,1080


In [29]:
# merge opponent attack strength
points = pd.merge(left = points,
                  right = team_strength_att[['id', 'strength_attack_away', 
                                             'strength_attack_home']],
                  how = 'left',
                  left_on = 'opponent_team',
                  right_on = 'id'
).drop(
    'id', axis = 1
).rename(
    columns={'strength_attack_away':'opp_att_strength_away', 
             'strength_attack_home':'opp_att_strength_home'}
)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength,opp_att_strength_away,opp_att_strength_home
0,1,Folarin Balogun,Arsenal,FWD,1,2,16,0,True,2023-08-12T12:00:00Z,...,0.00,0.00,45,0,59090,0,0,1120,1110,1050
1,1,Folarin Balogun,Arsenal,FWD,1,12,8,0,False,2023-08-21T19:00:00Z,...,0.00,0.00,45,-4744,63768,4959,9703,1080,1170,1140
2,1,Folarin Balogun,Arsenal,FWD,1,21,10,0,True,2023-08-26T14:00:00Z,...,0.00,0.00,45,-7682,58109,2507,10189,1140,1090,1090
3,1,Folarin Balogun,Arsenal,FWD,1,31,14,0,True,2023-09-03T15:30:00Z,...,0.00,0.00,44,-9696,49814,2558,12254,1240,1310,1150
4,1,Folarin Balogun,Arsenal,FWD,1,43,9,0,False,2023-09-17T15:30:00Z,...,0.00,0.00,44,-12786,37048,0,12786,1080,1120,1070


In [30]:
# assign correct home/away opponent attack strength for each fixture
def opp_att_strength(row):
    if row['was_home'] == False:
        return row['opp_att_strength_home']
    elif row['was_home'] == True:
        return row['opp_att_strength_away']
    else:
        return "Unknown"

points['opp_att_strength'] = points.apply(lambda row: opp_att_strength(row), axis = 1)

points = points.drop(['opp_att_strength_home','opp_att_strength_away'], axis = 1)

points.head()

,id_player,full_name,team,position,element,fixture,opponent_team,total_points,was_home,kickoff_time,...,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out,opp_def_strength,opp_att_strength
0,1,Folarin Balogun,Arsenal,FWD,1,2,16,0,True,2023-08-12T12:00:00Z,...,0.00,0.00,0.00,45,0,59090,0,0,1120,1110
1,1,Folarin Balogun,Arsenal,FWD,1,12,8,0,False,2023-08-21T19:00:00Z,...,0.00,0.00,0.00,45,-4744,63768,4959,9703,1080,1140
2,1,Folarin Balogun,Arsenal,FWD,1,21,10,0,True,2023-08-26T14:00:00Z,...,0.00,0.00,0.00,45,-7682,58109,2507,10189,1140,1090
3,1,Folarin Balogun,Arsenal,FWD,1,31,14,0,True,2023-09-03T15:30:00Z,...,0.00,0.00,0.00,44,-9696,49814,2558,12254,1240,1310
4,1,Folarin Balogun,Arsenal,FWD,1,43,9,0,False,2023-09-17T15:30:00Z,...,0.00,0.00,0.00,44,-12786,37048,0,12786,1080,1070


In [31]:
# get 20 top scoring players in all positions
gks = points.loc[points['position'] == 'GKP']
defs = points.loc[points['position'] == 'DEF']
mids = points.loc[points['position'] == 'MID']
fwds = points.loc[points['position'] == 'FWD']

top_20_gks = gks.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_defs = defs.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_mids = mids.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_fwds = fwds.groupby(
    ['element', 'full_name']
).agg(
    {'total_points':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head(20)

top_20_fwds.head()

,element,full_name,total_points
6,60,Ollie Watkins,59
50,355,Erling Haaland,55
49,343,Julián Álvarez,48
53,415,Alexander Isak,43
43,326,Carlton Morris,37


In [32]:
top_20_all_pos = pd.concat([top_20_gks, top_20_defs, top_20_mids, top_20_fwds], axis = 0)

# show the top 10 players by FPL points
top_20_all_pos.sort_values('total_points', ascending=False)[['full_name', 'total_points']].head(10)

,full_name,total_points
133,Mohamed Salah,59
6,Ollie Watkins,59
50,Erling Haaland,55
221,Son Heung-min,54
9,Bukayo Saka,53
67,Joachim Andersen,51
223,Jarrod Bowen,50
136,Kieran Trippier,49
49,Julián Álvarez,48
216,James Maddison,47


In [33]:
# select columns of interest
points_select = points[['id_player', 'full_name', 'team', 'position',
                        'total_points',
                        'minutes', 'goals_scored', 'assists', 'clean_sheets', 
                        'goals_conceded', 'own_goals',
                        'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
                        'expected_goals', 'expected_assists', 'expected_goal_involvements', 
                        'expected_goals_conceded', 'opp_att_strength', 'opp_def_strength']]

In [34]:
points_select['influence'].astype(float)

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5600    0.0
5601    0.0
5602    0.0
5603    0.0
5604    0.0
Name: influence, Length: 5605, dtype: float64

In [35]:
def last_5_player(df, player_id):
    ''' 
    get the mean stats for a given player_id over the last 5 fixtures
    prior to most recent fixture and the total points from the most 
    recent fixture. 
    
    assume dataframe is sorted from oldest to newest fixtures
    '''
    df = df[df['id_player'] == player_id]
    
    last_5 = df.tail(5)
    
    d = {'name': last_5['full_name'].iloc[0],
         'id': last_5['id_player'].iloc[0],
         'team': last_5['team'].iloc[0],
        'position': last_5['position'].iloc[0],
        'mean_points': last_5['total_points'].mean(),
        'mean_minutes': last_5['minutes'].mean(),
        'mean_goals_scored': last_5['goals_scored'].mean(),
        'mean_assists': last_5['assists'].mean(),
        'mean_clean_sheets': last_5['clean_sheets'].mean(),
        'mean_goals_conceded': last_5['goals_conceded'].mean(),
        'mean_own_goals': last_5['own_goals'].mean(),
        'mean_saves': last_5['saves'].mean(),
        'mean_bonus': last_5['bonus'].mean(),
        'mean_bps': last_5['bps'].mean(),
        'mean_influence': last_5['influence'].astype(float).mean(),
        'mean_creativity': last_5['creativity'].astype(float).mean(),
        'mean_threat': last_5['threat'].astype(float).mean(),
        'mean_ict': last_5['ict_index'].astype(float).mean(),
        'mean_xg': last_5['expected_goals'].astype(float).mean(),
        'mean_xa': last_5['expected_assists'].astype(float).mean(),
        'mean_xgi': last_5['expected_goal_involvements'].astype(float).mean(),
        'mean_xgc': last_5['expected_goals_conceded'].astype(float).mean(),
        'mean_opp_att': last_5['opp_att_strength'].mean(),
        'mean_opp_def': last_5['opp_def_strength'].mean()}
    
    last_5_mean = pd.DataFrame(data = d, index = [0])
    
    return last_5_mean

# test for Bukayo Saka (id = 13)
last_5_player(points_select, 13)

,name,id,team,position,mean_points,mean_minutes,mean_goals_scored,mean_assists,mean_clean_sheets,mean_goals_conceded,...,mean_influence,mean_creativity,mean_threat,mean_ict,mean_xg,mean_xa,mean_xgi,mean_xgc,mean_opp_att,mean_opp_def
0,Eddie Nketiah,13,Arsenal,FWD,2.4,74.4,0.0,0.2,0.6,0.6,...,3.36,7.78,15.0,2.46,0.104,0.03,0.134,0.68,1194.0,1190.0


In [36]:
def last_5_all(df):
    ''' get last mean stats for all players in df over the last 5 fixtures
    prior to most recent fixture and the total points from the most 
    recent fixture.
    '''
    last_5_all = pd.DataFrame() # empty dataframe
    for p in df['id_player'].unique():
        player_df = last_5_player(df, p)
        last_5_all = pd.concat([last_5_all, player_df])
    return last_5_all

In [37]:
last_5_df = last_5_all(points_select)

last_5_df.head()

,name,id,team,position,mean_points,mean_minutes,mean_goals_scored,mean_assists,mean_clean_sheets,mean_goals_conceded,...,mean_influence,mean_creativity,mean_threat,mean_ict,mean_xg,mean_xa,mean_xgi,mean_xgc,mean_opp_att,mean_opp_def
0,Folarin Balogun,1,Arsenal,FWD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0.000,1194.0,1190.0
0,Gabriel Fernando de Jesus,8,Arsenal,FWD,3.0,58.8,0.2,0.2,0.4,0.4,...,12.84,11.52,19.0,4.32,0.260,0.058,0.318,0.560,1194.0,1190.0
0,Eddie Nketiah,13,Arsenal,FWD,2.4,74.4,0.0,0.2,0.6,0.6,...,3.36,7.78,15.0,2.46,0.104,0.030,0.134,0.680,1194.0,1190.0
0,Keinan Davis,39,Aston Villa,FWD,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.00,0.000,0.000,0.000,0.000,1148.0,1149.0
0,Jhon Durán,44,Aston Villa,FWD,1.4,9.6,0.2,0.0,0.0,0.0,...,6.88,0.50,6.8,1.32,0.036,0.002,0.038,0.152,1148.0,1149.0


# Apply Models

## Midfield Players

In [44]:
# load pickled model
with open('models/mid_model_20230413.pkl', 'rb') as file:  
    mid_model = pickle.load(file)

/Users/elmeri/miniconda3/lib/python3.11/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [45]:
# apply the model
def predict_points(player_data, test_data, model) -> None:
    '''
    apply predictive model to the data
    '''
    prediction = player_data.assign(predicted=model.predict(test_data))
    
    return prediction[['name', 'team', 'position', 
                       'predicted']].sort_values('predicted', ascending=False).head(10).reset_index()

In [46]:
mid_data = last_5_df[last_5_df['position'] == 'MID']

mid_test = mid_data[['mean_ict', 'mean_xgi']]

predicted = predict_points(mid_data, mid_test, mid_model)

In [51]:
print(predicted)

   index                    name       team position  predicted
0      0           Mohamed Salah  Liverpool      MID   4.413557
1      0           Son Heung-min      Spurs      MID   3.633638
2      0      Abdoulaye Doucouré    Everton      MID   3.552677
3      0          James Maddison      Spurs      MID   3.477696
4      0             Bukayo Saka    Arsenal      MID   3.381006
5      0        Dejan Kulusevski      Spurs      MID   3.237842
6      0         Martin Ødegaard    Arsenal      MID   3.061377
7      0       James Ward-Prowse   West Ham      MID   3.018406
8      0  Bruno Borges Fernandes    Man Utd      MID   2.915789
9      0            Jarrod Bowen   West Ham      MID   2.839728


In [47]:
import panel as pn

In [48]:
# create panel dashboard
pn.extension(sizing_mode="stretch_width")

dash = pn.template.FastListTemplate(
    site="PredictFPL", 
    title="Next Gameweek Player Points Prediction", 
    sidebar=[], 
    main=[predicted,
         top_20_all_pos.sort_values('total_points', ascending=False)[['full_name', 'total_points']].head(10)],
    main_max_width="650px"
).servable();

In [49]:
dash

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [5899403792] DataFrame(DataFrame, sizing_mode='stretch_width')
    [5899412304] DataFrame(DataFrame, sizing_mode='stretch_width')